In [2]:
import requests
import time

def get_artist_mbid_for_album(album_title, artist_name):
    """
    Retrieves the artist MBID for a given album title and artist name
    using the MusicBrainz API.
    """
    base_url = "https://musicbrainz.org/ws/2/"
    headers = {"User-Agent": "MyMusicApp/1.0 (contact@example.com)"} # Replace with your app info

    # 1. Search for the release (album)
    search_url = f"{base_url}release/?query=release:{album_title}%20AND%20artist:{artist_name}&fmt=json"
    response = requests.get(search_url, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    search_data = response.json()

    release_id = None
    if search_data and 'releases' in search_data and search_data['releases']:
        # Assuming the first result is the most relevant
        release_id = search_data['releases'][0]['id']
    else:
        print(f"Album '{album_title}' by '{artist_name}' not found.")
        return None

    time.sleep(1) # Be respectful of the API rate limits

    # 2. Lookup the release to get artist information
    lookup_url = f"{base_url}release/{release_id}?inc=artist-credits&fmt=json"
    response = requests.get(lookup_url, headers=headers)
    response.raise_for_status()
    release_data = response.json()

    if release_data and 'artist-credit' in release_data and release_data['artist-credit']:
        # Assuming the first artist in the credit list is the primary artist
        artist_mbid = release_data['artist-credit'][0]['artist']['id']
        return artist_mbid
    else:
        print(f"Artist information not found for release ID: {release_id}")
        return None

# Example usage:
album_title = "One On One"
artist_name = "Earl Klugh"
artist_mbid = get_artist_mbid_for_album(album_title, artist_name)

if artist_mbid:
    print(f"The MBID for the artist of '{album_title}' is: {artist_mbid}")

The MBID for the artist of 'One On One' is: f4d2fc5a-cacb-47c8-b096-ee73864b1aa8
